In [ ]:
import numpy as np
import pandas as pd
import os
 
import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, SeparableConv2D, DepthwiseConv2D, AveragePooling2D,Activation,GlobalAveragePooling2D
from tensorflow.keras.applications import VGG16

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!unzip "/content/drive/My Drive/MRNet-v1.0"

Archive:  /content/drive/My Drive/MRNet-v1.0.zip
warning [/content/drive/My Drive/MRNet-v1.0.zip]:  4294967296 extra bytes at beginning or within zipfile
  (attempting to process anyway)
file #1:  bad zipfile offset (local header sig):  4294967296
  (attempting to re-compensate)
   creating: MRNet-v1.0/
  inflating: MRNet-v1.0/valid-abnormal.csv  
   creating: MRNet-v1.0/valid/
   creating: MRNet-v1.0/valid/axial/
  inflating: MRNet-v1.0/valid/axial/1139.npy  
  inflating: MRNet-v1.0/valid/axial/1138.npy  
  inflating: MRNet-v1.0/valid/axial/1249.npy  
  inflating: MRNet-v1.0/valid/axial/1248.npy  
  inflating: MRNet-v1.0/valid/axial/1207.npy  
  inflating: MRNet-v1.0/valid/axial/1213.npy  
  inflating: MRNet-v1.0/valid/axial/1171.npy  
  inflating: MRNet-v1.0/valid/axial/1165.npy  
  inflating: MRNet-v1.0/valid/axial/1159.npy  
  inflating: MRNet-v1.0/valid/axial/1158.npy  
  inflating: MRNet-v1.0/valid/axial/1164.npy  
  inflating: MRNet-v1.0/valid/axial/1170.npy  
  inflating: MRNet

In [ ]:
def read_data(data_type, scan_type):
  list = []
  for filename in sorted(os.listdir("/content/MRNet-v1.0/"+data_type+"/"+scan_type)):
    if(filename != ".DS_Store"):
      instance = np.load("/content/MRNet-v1.0/"+data_type+"/"+scan_type+"/" + filename)
      mid = len(instance) // 2
      instance = np.array([ instance[mid-1] , instance[mid] ,instance[mid+1]])
      list.append(instance)
  data = np.asarray(list)
  data = np.moveaxis(data, 1, 3)
  print(scan_type+data_type, data.shape)
  return data

def read_labels(data_type, scan_type):
  path = "/content/MRNet-v1.0/"+data_type+"-"+scan_type + ".csv"
  df=pd.read_csv(path, sep=',',header=None)
  labels = df.values
  labels = labels[:,1]
  print(scan_type+data_type, labels.shape)
  return labels


def read_data_singles(data_type, scan_type, label_scan_type):
  list = []
  path = "/content/MRNet-v1.0/"+data_type+"-"+label_scan_type + ".csv"
  df=pd.read_csv(path, sep=',',header=None)
  labels = df.values
  labels = labels[:,1]
  single_labels = []
  i=0
  for filename in sorted(os.listdir("/content/MRNet-v1.0/"+data_type+"/"+scan_type)):
    if(filename != ".DS_Store"):
      instance = np.load("/content/MRNet-v1.0/"+data_type+"/"+scan_type+"/" + filename)
      for image in instance:
        list.append(np.array(image))
        single_labels.append(labels[i])
      i += 1
  data = np.asarray(list)
  data = data[:, :, :, np.newaxis]
  single_labels = np.asarray(single_labels)
  print("single")
  print(scan_type+data_type, data.shape)
  print(label_scan_type+data_type, single_labels.shape)
  return data, single_labels

# single_sagittal_train_x, single_saggital_abnormal_train_y = read_data_singles("train","sagittal","abnormal")
# single_sagittal_test_x, single_saggital_abnormal_test_y = read_data_singles("valid","sagittal","abnormal")

print("Data: ")

sagittal_train_x = read_data("train","sagittal")
axial_train_x = read_data("train","axial")
coronal_train_x = read_data("train","coronal")

sagittal_test_x = read_data("valid","sagittal")
axial_test_x = read_data("valid","axial")
coronal_test_x = read_data("valid","coronal")

print("Labels: ")
abnormal_train_y = read_labels('train','abnormal')
acl_train_y = read_labels('train','acl')
meniscus_train_y = read_labels('train','meniscus')

abnormal_test_y = read_labels('valid','abnormal')
acl_test_y = read_labels('valid','acl')
meniscus_test_y = read_labels('valid','meniscus')

Data: 
sagittaltrain (1130, 256, 256, 3)
axialtrain (1130, 256, 256, 3)
coronaltrain (1130, 256, 256, 3)
sagittalvalid (120, 256, 256, 3)
axialvalid (120, 256, 256, 3)
coronalvalid (120, 256, 256, 3)
Labels: 
abnormaltrain (1130,)
acltrain (1130,)
meniscustrain (1130,)
abnormalvalid (120,)
aclvalid (120,)
meniscusvalid (120,)


In [ ]:
def build_model():
  vgg = VGG16(include_top=False, weights='imagenet', input_tensor=None, input_shape=(256,256,3), pooling=None)
  for layer in vgg.layers:
    layer.trainable = False
  x = vgg.output
  x = Flatten()(x)
  x = Dense(4096, activation='relu')(x)
  x = Dropout(0.5)(x)
  x = Dense(4096, activation='relu')(x)
  x = Dropout(0.5)(x)
  x = Dense(1, activation='sigmoid')(x)

  vgg_model = Model(inputs = vgg.input, outputs = x)
  vgg_model.summary()
  vgg_model.compile(loss='binary_crossentropy', optimizer= "adam", metrics=['accuracy'])

  return vgg_model

def build_three_binary_models(train_y, test_y):
  sagittal_model = build_model()
  sagittal_model.fit(sagittal_train_x, train_y, batch_size=16, epochs=20, validation_split=0.1, verbose=1,shuffle=True)
  sagittal_model.evaluate(sagittal_test_x, test_y, batch_size=16)

  axial_model = build_model()
  axial_model.fit(axial_train_x, train_y, batch_size=16, epochs=20, validation_split=0.1, verbose=1,shuffle=True)
  axial_model.evaluate(axial_test_x, test_y, batch_size=16)

  coronal_model = build_model()
  coronal_model.fit(coronal_train_x, train_y, batch_size=16, epochs=20, validation_split=0.1, verbose=1,shuffle=True)
  coronal_model.evaluate(coronal_test_x, test_y, batch_size=16)
  return sagittal_model, axial_model, coronal_model

In [ ]:
#Instantiate an empty model
model = Sequential()

model.add(Conv2D(filters=96, input_shape=(256,256,3), kernel_size=(11,11), activation='relu', strides=(4,4), padding='valid'))# Max Pooling
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))

model.add(Conv2D(filters=256, kernel_size=(5,5), activation='relu', strides=(1,1), padding='same'))
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))

model.add(Conv2D(filters=384, kernel_size=(3,3),  activation='relu', strides=(1,1), padding='same'))
model.add(Activation('relu'))

model.add(Conv2D(filters=384, kernel_size=(3,3),  activation='relu', strides=(1,1), padding='same'))
model.add(Activation('relu'))

model.add(Conv2D(filters=256, kernel_size=(3,3),  activation='relu', strides=(1,1), padding='same'))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
model.add(GlobalAveragePooling2D())
model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.summary()

# Compile the model
# model.compile(loss=keras.losses.categorical_crossentropy, optimizer='adam', metrics=['accuracy'])

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_106 (Conv2D)          (None, 62, 62, 96)        34944     
_________________________________________________________________
max_pooling2d_62 (MaxPooling (None, 30, 30, 96)        0         
_________________________________________________________________
conv2d_107 (Conv2D)          (None, 30, 30, 256)       614656    
_________________________________________________________________
max_pooling2d_63 (MaxPooling (None, 14, 14, 256)       0         
_________________________________________________________________
conv2d_108 (Conv2D)          (None, 14, 14, 384)       885120    
_________________________________________________________________
activation_85 (Activation)   (None, 14, 14, 384)       0         
_________________________________________________________________
conv2d_109 (Conv2D)          (None, 14, 14, 384)     

In [ ]:
#Abnormal models
abnormal_sagittal_model, abnormal_axial_model, abnormal_coronal_model = build_three_binary_models(abnormal_train_y, abnormal_test_y)

In [ ]:

abnormal_axial_model.save_weights("drive/My Drive/models/ab_ax.h5")
print("Saved model to disk")

abnormal_coronal_model.save_weights("drive/My Drive/models/ab_cr.h5")
print("Saved model to disk")

abnormal_sagittal_model.save_weights("drive/My Drive/models/ab_sag.h5")
print("Saved model to disk")

Saved model to disk
Saved model to disk
Saved model to disk


In [ ]:
#Acl models
acl_sagittal_model, acl_axial_model, acl_coronal_model = build_three_binary_models(acl_train_y, acl_test_y)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0     

In [ ]:

acl_axial_model.save_weights("drive/My Drive/models/acl_ax.h5")
print("Saved model to disk")

acl_coronal_model.save_weights("drive/My Drive/models/acl_cr.h5")
print("Saved model to disk")


acl_sagittal_model.save_weights("drive/My Drive/models/acl_sag.h5")
print("Saved model to disk")

Saved model to disk
Saved model to disk
Saved model to disk


In [ ]:
#Meniscus models
meniscus_sagittal_model, meniscus_axial_model, meniscus_coronal_model = build_three_binary_models(meniscus_train_y, meniscus_test_y)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0     

In [ ]:

meniscus_axial_model.save_weights("drive/My Drive/models/men_ax.h5")
print("Saved model to disk")


meniscus_coronal_model.save_weights("drive/My Drive/models/men_cr.h5")
print("Saved model to disk")


meniscus_sagittal_model.save_weights("drive/My Drive/models/men_sag.h5")
print("Saved model to disk")



Saved model to disk
Saved model to disk
Saved model to disk
4/4 [==============================] - 0s 76ms/step - loss: 1.2218 - accuracy: 0.6500


[1.2217838764190674, 0.6499999761581421]

4/4 [==============================] - 0s 74ms/step - loss: 10.1724 - accuracy: 0.5833


[10.172375679016113, 0.5833333134651184]

In [ ]:
import scipy
from scipy import stats, optimize, interpolate

#majority vote ensamble
def getPredictions(models,data):
  labels=[]
  for m in range(len(models)):
    predicts = np.round(models[m].predict(data[m]))
    labels.append(predicts)

  labels = np.array(labels)
  labels = stats.mode(labels, axis=0)[0]
  labels = np.squeeze(labels)
  return labels

#calculate acuuracy
def eval(predictions,ground):
  n=0
  for i in range(len(labels)):
    if(predictions[i]==ground[i]):
      n+=1
  return (n/len(ground))



In [ ]:
  #load models
acl=[]
acl_ax=Model()
acl_ax=build_model()
acl_ax.load_weights('drive/My Drive/models/acl_ax.h5')
acl.append(acl_ax)
acl_sag=Model()
acl_sag=build_model()
acl_sag.load_weights('drive/My Drive/models/acl_sag.h5')
acl.append(acl_sag)
acl_cr=Model()
acl_cr=build_model()
acl_cr.load_weights('drive/My Drive/models/acl_cr.h5')
acl.append(acl_cr)

men=[]
men_ax=Model()
men_ax=build_model()
men_ax.load_weights('drive/My Drive/models/men_ax.h5')
men.append(men_ax)
men_sag=Model()
men_sag=build_model()
men_sag.load_weights('drive/My Drive/models/men_sag.h5')
men.append(men_sag)
men_cr=Model()
men_cr=build_model()
men_cr.load_weights('drive/My Drive/models/men_cr.h5')
men.append(men_cr)

ab=[]
ab_ax=Model()
ab_ax=build_model()
ab_ax.load_weights('drive/My Drive/models/ab_ax.h5')
ab.append(ab_ax)
ab_sag=Model()
ab_sag=build_model()
ab_sag.load_weights('drive/My Drive/models/ab_sag.h5')
ab.append(ab_sag)
ab_cr=Model()
ab_cr=build_model()
ab_cr.load_weights('drive/My Drive/models/ab_cr.h5')
ab.append(ab_cr)



Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0   

In [ ]:
data=[]
data.append(axial_test_x)
data.append(sagittal_test_x)
data.append(coronal_test_x)


acl_pred=getPredictions(acl,data)
print("ACL acuuracy:",eval(acl_pred,acl_test_y))
ab_pred=getPredictions(ab,data)
print("Abnormal acuuracy:",eval(ab_pred,abnormal_test_y))
men_pred=getPredictions(men,data)
print("Meniscus acuuracy:",eval(men_pred,meniscus_test_y))

ACL acuuracy: 0.6916666666666667
Abnormal acuuracy: 0.8166666666666667
Meniscus acuuracy: 0.6583333333333333
